<a href="https://colab.research.google.com/github/felolivee/DVA-NYC_Congestion/blob/Ashley/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Data

Sometimes method 1 doesn't work for me, so I added an alternative. Both require uploading your Kaggle API token, which you can get from https://www.kaggle.com/settings. Also make sure your runtime is set to the standard CPU so it executes faster!

### Method 1:

In [ ]:
#download from Kaggle
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d jeffsinsel/nyc-fhvhv-data

#unzip files in nyc-fhvhv-data from Kaggle to content folder
! unzip nyc-fhvhv-data.zip

### Method 2:

In [ ]:
#download from Kaggle
! pip3 install -q kagglehub
import kagglehub
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
path = kagglehub.dataset_download("jeffsinsel/nyc-fhvhv-data")
print("Path to dataset:", path)

# move dataset to content folder
!mv {path}/* /content

Saving kaggle.json to kaggle.json


100%|██████████| 17.8G/17.8G [03:03<00:00, 105MB/s]

Extracting files...


Path to dataset: /root/.cache/kagglehub/datasets/jeffsinsel/nyc-fhvhv-data/versions/4


## Extract Features

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
import glob

# initialize spark
spark = SparkSession.builder.appName("NYC_Rides").getOrCreate()

# list of all parquet files from content folder
parquet_files = glob.glob('/content/*.parquet')

removed_summary = {}
combined_df = None

for file in parquet_files:
    # read parquet files get features
    df = spark.read.parquet(file).select(
        "Pickup_datetime", "DropOff_datetime", "PULocationID", "DOLocationID",
        "base_passenger_fare", "trip_miles", "tips", "driver_pay", "trip_time",
        "Hvfhs_license_num", "congestion_surcharge"
    )

    initial_count = df.count()

    # Clean data by filtering out null and invalid values
    df_clean = df.filter(
        (F.col("Pickup_datetime").isNotNull()) &
        (F.col("DropOff_datetime").isNotNull()) &
        (F.col("PULocationID").isNotNull()) &
        (F.col("DOLocationID").isNotNull()) &
        (F.col("base_passenger_fare").isNotNull()) &
        (F.col("trip_miles").isNotNull()) &
        (F.col("tips").isNotNull()) &
        (F.col("driver_pay").isNotNull()) &
        (F.col("trip_time").isNotNull()) &
        (F.col("Hvfhs_license_num").isNotNull()) &
        (F.col("congestion_surcharge").isNotNull()) &

        # Ensure numeric columns have valid positive values
        (F.col("base_passenger_fare") > 0) &
        (F.col("trip_miles") > 0) &
        (F.col("driver_pay") > 0) &
        (F.col("trip_time") > 0)
    )

    clean_count = df_clean.count()
    removed_summary[file] = initial_count - clean_count

    # combine clean DFs into one big DF
    if combined_df is None:
        combined_df = df_clean
    else:
        combined_df = combined_df.union(df_clean)

# check how many of which file were removed
print("Removal Summary:", removed_summary)

Removal Summary: {'/content/fhvhv_tripdata_2019-05.parquet': 2197902, '/content/fhvhv_tripdata_2019-12.parquet': 2247128, '/content/fhvhv_tripdata_2021-06.parquet': 155107, '/content/fhvhv_tripdata_2021-08.parquet': 130783, '/content/fhvhv_tripdata_2022-04.parquet': 82929, '/content/fhvhv_tripdata_2020-06.parquet': 110413, '/content/fhvhv_tripdata_2021-01.parquet': 103408, '/content/fhvhv_tripdata_2019-09.parquet': 2193661, '/content/fhvhv_tripdata_2022-11.parquet': 323983, '/content/fhvhv_tripdata_2019-11.parquet': 2343369, '/content/fhvhv_tripdata_2022-01.parquet': 77981, '/content/fhvhv_tripdata_2019-08.parquet': 4465149, '/content/fhvhv_tripdata_2020-08.parquet': 135659, '/content/fhvhv_tripdata_2019-07.parquet': 2783564, '/content/fhvhv_tripdata_2021-05.parquet': 142950, '/content/fhvhv_tripdata_2020-01.parquet': 2120334, '/content/fhvhv_tripdata_2021-04.parquet': 143913, '/content/fhvhv_tripdata_2020-07.parquet': 188921, '/content/fhvhv_tripdata_2021-09.parquet': 134529, '/conten

### Data Preview (Optional)

In [ ]:
# see schema
combined_df.printSchema()

root
 |-- Pickup_datetime: timestamp_ntz (nullable = true)
 |-- DropOff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- Hvfhs_license_num: string (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [ ]:
# calculate how many total rows of data were removed
tot = 0
for key,val in removed_summary.items():
  tot += int(val)

count = combined_df.count()
percent_removed = tot/count * 100

print("{}% was removed from a total of {} rides in 46 months".format(percent_removed, count))

5.2355475054961005% was removed from a total of 708208434 rides in 46 months


In [ ]:
# aggregate features for Tableau file

add_hour = combined_df.withColumn("hour", F.date_format(F.col("Pickup_datetime"), "k"))

tableau_df = add_hour.groupBy(["PULocationID", "pickup_hour", "Hvfhs_license_num"]).agg(F.count("PULocationID").alias("num_rides"),F.mean("trip_miles").alias("avg_trip_miles"), F.mean("driver_pay").alias("avg_driver_pay"))


tableau_df = tableau_df.withColumn("rideType", F.when(F.col("Hvfhs_license_num") == "HV0002", "Juno")
     .when(F.col("Hvfhs_license_num") == "HV0003", "Uber")
     .when(F.col("Hvfhs_license_num") == "HV0004", "Via")
     .when(F.col("Hvfhs_license_num") == "HV0005", "Lyft"))

In [ ]:
tableau_df.write.option('header', True).csv('/content/drive/MyDrive/dva_project.csv')
